In [1]:
import openai
import configparser

conf = configparser.ConfigParser()
conf.read("../config.ini")
openai.api_key = conf.get("Openai", "api_key")


In [5]:
all_models = openai.Model.list()
all_models = [model.get('id') for model in all_models.get("data")]
print(all_models, len(all_models))

['babbage', 'text-davinci-003', 'davinci', 'text-davinci-edit-001', 'babbage-code-search-code', 'text-similarity-babbage-001', 'code-davinci-edit-001', 'text-davinci-001', 'ada', 'babbage-code-search-text', 'babbage-similarity', 'gpt-3.5-turbo-16k-0613', 'code-search-babbage-text-001', 'text-curie-001', 'gpt-3.5-turbo-0301', 'gpt-3.5-turbo-16k', 'code-search-babbage-code-001', 'text-ada-001', 'text-similarity-ada-001', 'text-davinci-002', 'curie-instruct-beta', 'ada-code-search-code', 'ada-similarity', 'code-search-ada-text-001', 'text-search-ada-query-001', 'davinci-search-document', 'ada-code-search-text', 'text-search-ada-doc-001', 'davinci-instruct-beta', 'text-similarity-curie-001', 'code-search-ada-code-001', 'ada-search-query', 'text-search-davinci-query-001', 'curie-search-query', 'davinci-search-query', 'babbage-search-document', 'ada-search-document', 'text-search-curie-query-001', 'text-search-babbage-doc-001', 'whisper-1', 'curie-search-document', 'text-search-curie-doc-001

In [6]:
openai.Model.retrieve("gpt-3.5-turbo")  # 获取单个模型详细信息

<Model model id=gpt-3.5-turbo at 0x12446d670> JSON: {
  "id": "gpt-3.5-turbo",
  "object": "model",
  "created": 1677610602,
  "owned_by": "openai",
  "permission": [
    {
      "id": "modelperm-ILldYg889l2lED1b99xWEfey",
      "object": "model_permission",
      "created": 1689885359,
      "allow_create_engine": false,
      "allow_sampling": true,
      "allow_logprobs": true,
      "allow_search_indices": false,
      "allow_view": true,
      "allow_fine_tuning": false,
      "organization": "*",
      "group": null,
      "is_blocking": false
    }
  ],
  "root": "gpt-3.5-turbo",
  "parent": null
}

## OpenAI上下文补全Completions与ChatCompletions API

In [13]:
# Completion
conversation = openai.Completion.create(
    model='text-davinci-003',
    prompt='You are a translation assistant',
    max_tokens=10,
    temperature=0.2,
    stream=False
)

In [20]:
text = conversation["choices"][0]['text']
print(text)



As a translation assistant, your job is


In [17]:
conversation_2 = openai.Completion.create(
    model='text-davinci-003',
    prompt=text,
    max_tokens=10,
    temperature=0.2,
    stream=False
)

In [19]:
text2 = conversation_2["choices"][0]['text']
print(text2)

 to help the translator with the translation process. You


In [36]:
# ChatCompletion
message = [
    {"role": "system", "content": "You are a translation assistant, and you name is Macy, a kindness young lady."},
    {"role": "user", "content": "Hi Macy! I am glad to see you."}
]
conversation_3 = openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=message,
    max_tokens=50,
    temperature=0.2,
    stream=True
)
for event in conversation_3:
    if event['choices'][0]['finish_reason'] != 'stop':
        print(event['choices'][0]['delta']['content'], end='')

Hello! I'm delighted to see you too. How can I assist you today?